In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests

In [2]:
#parseamos el html
url = "https://www.gutenberg.org/browse/scores/top1000.php#books-last1"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [3]:
links = []
#Primero buscamos todas las listas ('ol'), luego seleccionamos la lista que tiene los libros mas populares de los ultimos 30 días
#Luego bucamos todos los elementos links ('a') de la lista para extraer y almacenar el 'href' de cada elemento.
#El href contiene la url de cada libro.
ol = soup.find_all('ol')
for a in ol[4].find_all('a'):
  link = a.get('href')
  links.append(link)
len(links)

1000

In [4]:
titles = []
generos = []
descripciones = []
authors = []

for link in links: 
  url = "https://www.gutenberg.org" + link #Visitamos todas las urls
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  secciones_tabla = soup.find('table', {'class': 'bibrec'}).find_all('tr') #Buscamos la tabla donde tiene la informacion del libro
  title = ''
  author = ''
  genero = ''
  descripcion = ''
  for seccion in secciones_tabla:
    categoria = seccion.find_all('th') 
    contenido = seccion.find_all('td')
    if len(categoria) > 0:
      if categoria[0].get_text() == 'Title':
        title = title + contenido[0].get_text()
      if categoria[0].get_text() == 'Author':
        author = author + contenido[0].get_text()
      if categoria[0].get_text() == 'Subject':
        genero = genero + contenido[0].get_text()
      if categoria[0].get_text() == 'Summary':
        descripcion = descripcion + contenido[0].get_text()
  titles.append(title)
  generos.append(genero)
  descripciones.append(descripcion)
  authors.append(author)


In [5]:
#Limpiamos los datos despues del scraping
generos_normalizada = []
titles_normalizada = []
descripciones_normalizada = []
authors_normalizada = []
for elemento in generos:
    elemento = elemento.replace('\n', ' ').replace('--', '').replace(',', '').replace('.', '').replace('(', '').replace(')', '')
    elemento = re.sub(r'\s{2,}', ' ', elemento)
    generos_normalizada.append(elemento)

for title in titles:
    title = title.replace('\n', '').replace(',', '').replace(';', '').replace('.', '')
    titles_normalizada.append(title)

for descripcion in descripciones:
    descripcion = descripcion.replace('\n', '').replace(';', '')
    descripciones_normalizada.append(descripcion)

for author in authors:
    author = author.replace('\n', '').replace(',', ' ')
    authors_normalizada.append(author)

In [6]:
df = pd.DataFrame({
    'Title': titles_normalizada,
    'Genre': generos_normalizada,
    'Description': descripciones_normalizada,
    'Author': authors_normalizada
})


In [7]:
df.to_csv('./databases/libros_bd.csv', index=False)